La voiture : étude d'un fait social

Par Alexandre Pakin et Gilles Massoulié

In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Introduction

L’usage de la voiture constitue un objet central d’analyse en sociologie, tant il structure les mobilités quotidiennes, les trajectoires sociales et les rapports aux territoires en France. Loin d’être un simple outil de déplacement, l’automobile peut être appréhendée comme un fait social, révélateur d’inégalités de classe, de genre, d’âge et de territoire.

Cette étude a été motivée par une [Vidéo de vulgarisation sociologique sur l’automobile](https://www.youtube.com/watch?v=tR6aujaUh_g)
 proposant une synthèse critique de plusieurs travaux récents sur l’automobile et ses usages sociaux. En mettant en évidence la dimension profondément inégalitaire de la conduite et de la possession d’un véhicule, cette vidéo invite à revenir aux données empiriques afin de reproduire et discuter certaines conclusions à partir de sources ouvertes. Le présent projet s’inscrit ainsi dans une démarche de réanalyse quantitative, visant à confronter des hypothèses sociologiques établies aux données disponibles.

La littérature récente permet de dégager plusieurs axes majeurs. D’une part, des travaux généraux de sociologie de l’automobile, comme [Sociologie de l’automobile](https://www.cairn.info/sociologie-de-l-automobile--9782707196433.htm) de Jean-Pierre Orfeuil et Frédéric Héran (2022) ou [Idées reçues sur l’automobile](https://www.cairn.info/idees-recues-sur-l-automobile--9782715405666.htm) de Frédéric Héran (2021), proposent un cadre théorique permettant de déconstruire les représentations dominantes associées à la voiture. D’autre part, plusieurs études mettent en évidence des usages différenciés selon les caractéristiques sociales des individus. Les dimensions d’âge et de genre sont notamment abordées dans des travaux portant sur la jeunesse automobile ou sur les inégalités et comportements genrés au volant, tandis que d’autres recherches analysent l’automobile comme un marqueur de position sociale et un révélateur des inégalités de classe, notamment à travers les politiques de mobilité et de sécurité routière.

Dans le prolongement de ces travaux, ce projet vise à analyser empiriquement l’usage de la voiture en France à partir de données issues de trois API publiques (data.gouv.fr, INSEE et transport.data.gouv.fr). L’objectif est d’identifier des régularités statistiques et des inégalités sociales liées à l’automobile, tout en illustrant l’apport des outils de programmation en Python pour l’analyse quantitative de phénomènes sociaux.

Sommaire (à faire)

Installation

In [2]:
!pip install -r requirements.txt

# modules 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
import re
from io import BytesIO
from urllib import request
import requests
from functools import reduce
import requests


# Fonctions
from modules import module_BAAC
from modules import module_region
from modules import graph_baac

Préparation des données

1. Adresses

Toutes les données utilisées dans ce projet sont récupérées directement en ligne, principalement via des interfaces de programmation (API) publiques. Ces API permettent un accès automatisé et reproductible aux données, ce qui s’inscrit pleinement dans une démarche de science des données appliquée aux sciences sociales.

In [12]:
# API Bases de données annuelles des accidents corporels de la circulation routière – 2005 à 2024 (gouvernement.fr)
BAAC_url = "https://www.data.gouv.fr/api/1/datasets/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2024/"

# URLs d'export CSV : consommation quotidienne d’alcool (Odissé - Santé Publique France)
URL_CONSO = "https://odisse.santepubliquefrance.fr/explore/dataset/alcool-consommation-quotidienne-region/download/?format=csv"

# URLs d'export CSV : alcoolisation ponctuelle importante (Odissé - Santé Publique France)
URL_API   = "https://odisse.santepubliquefrance.fr/explore/dataset/alcool-alcoolisation-ponctuelle-importante-api-mensuel-et-hebdomadaire-region/download/?format=csv"

# URLs d'export .XLSX : impoosition par région (gouvernement.fr)
url_impot = "https://www.impots.gouv.fr/node/25517"   # page with the file

2. Liste des accidents

(à compléter, chaque ligne est un accident)

In [13]:
BAAC_resources = module_BAAC.get_resources(BAAC_url)
pd.set_option("display.max_rows", None)
pd.options.display.max_colwidth = 300
BAAC_resources[["description", "url"]].head(10)

,description,url
0,Liste des véhicules immatriculés (hors vélos et engins de déplacement personnel) impliqués dans les accidents corporels issus des BAAC (Bulletin d’Analyse des Accidents Corporels de la circulation) relevés par les forces de l’ordre en 2024.\n,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2024/20251107-100210/2024.csv
1,Liste des véhicules immatriculés (hors vélos et engins de déplacement personnel) impliqués dans les accidents corporels issus des BAAC (Bulletin d’Analyse des Accidents Corporels de la circulation) relevés par les forces de l’ordre en 2023.\n,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2024/20251021-142813/2023.csv
2,Table des caractéristiques des accidents corporels de la circulation en 2024 en France (France métropolitaine + DOM + COM +NC).\n,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2024/20251021-115900/caract-2024.csv
3,Table des lieux des accidents corporels de la circulation en 2024 en France (France métropolitaine + DOM + COM +NC).\n,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2024/20251021-115812/lieux-2024.csv
4,Table des véhicules impliqués dans les accidents corporels de la circulation en 2024 en France (France métropolitaine + DOM + COM +NC).\n,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2024/20251107-100240/vehicules-2024.csv
5,Table des usagers impliqués dans les accidents corporels de la circulation en 2024 en France (France métropolitaine + DOM + COM +NC).\n,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2024/20251021-115506/usagers-2024.csv
6,Liste des véhicules immatriculés (hors vélos et engins de déplacement personnel) impliqués dans les accidents corporels issus des BAAC (Bulletin d’Analyse des Accidents Corporels de la circulation) relevés par les forces de l’ordre en 2022.\n,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2024/20251021-142722/2022.csv
7,Table des usagers impliqués dans les accidents corporels de la circulation en 2023 en France (France métropolitaine + DOM + COM +NC).,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2023/20241023-153328/usagers-2023.csv
8,Table des véhicules impliqués dans les accidents corporels de la circulation en 2023 en France (France métropolitaine + DOM + COM +NC).,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2023/20241023-153253/vehicules-2023.csv
9,Table des lieux des accidents corporels de la circulation en 2023 en France (France métropolitaine + DOM + COM +NC).,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2023/20241023-153219/lieux-2023.csv


On a sélectionné les catégories suivantes (à rédiger), toutes les tables sauf la première qui est une liste des véhicules immatriculés dans les accidents ont en lien la variable num_acc, donc on les fusionne pour obtenir le dataframe BAAC_table.
Il faudra qd mm utiliser le premier dataframe car il est sûrement intéressant.

In [14]:
selected_BAAC_table = module_BAAC.select_baac_tables(BAAC_resources)
selected_BAAC_table

,description,url,table,year
0,Table des caractéristiques des accidents corporels de la circulation en 2024 en France (France métropolitaine + DOM + COM +NC).\n,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2024/20251021-115900/caract-2024.csv,caracteristiques,2024
1,Table des lieux des accidents corporels de la circulation en 2024 en France (France métropolitaine + DOM + COM +NC).\n,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2024/20251021-115812/lieux-2024.csv,lieux,2024
2,Table des véhicules impliqués dans les accidents corporels de la circulation en 2024 en France (France métropolitaine + DOM + COM +NC).\n,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2024/20251107-100240/vehicules-2024.csv,vehicules,2024
3,Table des usagers impliqués dans les accidents corporels de la circulation en 2024 en France (France métropolitaine + DOM + COM +NC).\n,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2024/20251021-115506/usagers-2024.csv,usagers,2024
4,Table des usagers impliqués dans les accidents corporels de la circulation en 2023 en France (France métropolitaine + DOM + COM +NC).,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2023/20241023-153328/usagers-2023.csv,usagers,2023
5,Table des véhicules impliqués dans les accidents corporels de la circulation en 2023 en France (France métropolitaine + DOM + COM +NC).,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2023/20241023-153253/vehicules-2023.csv,vehicules,2023
6,Table des lieux des accidents corporels de la circulation en 2023 en France (France métropolitaine + DOM + COM +NC).,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2023/20241023-153219/lieux-2023.csv,lieux,2023
7,Table des caractéristiques des accidents corporels de la circulation en 2023 en France (France métropolitaine + DOM + COM +NC).,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2023/20241028-103125/caract-2023.csv,caracteristiques,2023
8,Table des usagers impliqués dans les accidents corporels de la circulation en 2022 en France (France métropolitaine + DOM + COM +NC).,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2021/20231005-094229/usagers-2022.csv,usagers,2022
9,Table des véhicules impliqués dans les accidents corporels de la circulation en 2022 en France (France métropolitaine + DOM + COM +NC).,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2021/20231005-094147/vehicules-2022.csv,vehicules,2022


Le code peut prendre longtemps à tourner (plus de 10mins) en fonction de la connexion internet.

In [16]:
BAAC_table = module_BAAC.build_baac_dataframe(selected_BAAC_table)
BAAC_table.head(10)

c:\Users\pakin\Desktop\boulot\info\GitHub\ENSAE-2A-S1\modules\module_BAAC.py:91: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  def _load(u):
c:\Users\pakin\Desktop\boulot\info\GitHub\ENSAE-2A-S1\modules\module_BAAC.py:91: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  def _load(u):


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 208: invalid continuation byte

In [123]:
for i in range(2020, 2025):
    globals()[f'BAAC_table_{i}'] = BAAC_table.loc[BAAC_table['year'] == i]

len(BAAC_table_2024['Num_Acc'])

5621008

In [110]:
BAAC_table = BAAC_table.loc[BAAC_table['year'] == 2024]
len(BAAC_table.head())

5

In [111]:
BAAC_table[['lat','long']].head()

,lat,long
5314184,"47,56277000","6,75832000"
5314185,"47,56277000","6,75832000"
5314186,"47,56277000","6,75832000"
5314187,"47,56277000","6,75832000"
5314188,"47,02109000","4,83755000"


blabla 2e dataframe

In [120]:
# Chargement en dataframes
df_conso = pd.read_csv(URL_CONSO, sep=';')
df_api   = pd.read_csv(URL_API, sep=';')
df_conso.head()

,annee,reg,reglib,sexe,alc_quo_v,alc_quo_icb,alc_quo_ich,alc_quo_typo
0,2000,32,Hauts-de-France,Femmes,11.4334,8.8527,14.6454,B
1,2000,75,Nouvelle Aquitaine,Femmes,14.0312,11.4041,17.1462,B
2,2000,27,Bourgogne et Franche-Comté,Hommes,27.4403,21.9954,33.6515,B
3,2000,28,Normandie,Hommes et Femmes,15.6665,12.6850,19.1946,A
4,2000,32,Hauts-de-France,Hommes et Femmes,21.1408,18.4634,24.0918,B


In [8]:
response = requests.get(url_impot)
df1 = pd.read_excel(BytesIO(response.content), sheet_name='regional_nombres')
df2 = pd.read_excel(BytesIO(response.content), sheet_name='regional_montants')
df1

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(nombres en milliers)
1,RÉGIONS,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022.0,2023
2,ILE-DE-FRANCE,6770.6,6809.9,6831.3,6907.2,6978.2,7152.0,7241.9,7247.0,7310.0,7428
3,CENTRE VAL DE LOIRE,1448.5,1455.2,1464.0,1481.5,1476.3,1494.1,1506.0,1520.0,1531.0,1543
4,BOURGOGNE FRANCHE COMTE,1586.4,1590.8,1592.0,1598.6,1598.1,1618.9,1634.8,1643.0,1660.0,1667
5,NORMANDIE,1857.8,1865.0,1868.0,1877.3,1881.6,1912.9,1936.2,1954.0,1973.0,1987
6,HAUTS DE FRANCE,3261.9,3273.2,3278.0,3302.9,3297.6,3345.1,3383.5,3409.0,3434.0,3455
7,GRAND EST,3054.6,3065.4,3075.2,3097.7,3120.9,3155.6,3182.2,3194.0,3213.0,3234
8,PAYS DE LA LOIRE,2004.3,2026.3,2045.4,2075.8,2095.9,2143.8,2181.1,2210.0,2244.0,2269
9,BRETAGNE,1824.8,1842.4,1857.5,1884.9,1894.8,1935.9,1971.5,2006.0,2036.0,2062


In [9]:
df1_fixed = module_region.excel_fix(df1)
df2_fixed = module_region.excel_fix(df2)
df1_fixed

,region_id,region,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,11,ÎLE-DE-FRANCE,6770.6,6809.9,6831.3,6907.2,6978.2,7152.0,7241.9,7247.0,7310.0,7428
1,24,CENTRE-VAL DE LOIRE,1448.5,1455.2,1464.0,1481.5,1476.3,1494.1,1506.0,1520.0,1531.0,1543
2,27,BOURGOGNE-FRANCHE-COMTÉ,1586.4,1590.8,1592.0,1598.6,1598.1,1618.9,1634.8,1643.0,1660.0,1667
3,28,NORMANDIE,1857.8,1865.0,1868.0,1877.3,1881.6,1912.9,1936.2,1954.0,1973.0,1987
4,32,HAUTS DE FRANCE,3261.9,3273.2,3278.0,3302.9,3297.6,3345.1,3383.5,3409.0,3434.0,3455
5,44,GRAND EST,3054.6,3065.4,3075.2,3097.7,3120.9,3155.6,3182.2,3194.0,3213.0,3234
6,52,PAYS DE LA LOIRE,2004.3,2026.3,2045.4,2075.8,2095.9,2143.8,2181.1,2210.0,2244.0,2269
7,53,BRETAGNE,1824.8,1842.4,1857.5,1884.9,1894.8,1935.9,1971.5,2006.0,2036.0,2062
8,75,NOUVELLE AQUITAINE,3404.1,3433.7,3463.4,3518.7,3530.4,3604.3,3663.8,3715.0,3766.0,3805
9,76,OCCITANIE,3311.9,3338.8,3367.1,3424.1,3456.4,3539.2,3604.5,3664.0,3732.0,3778


In [10]:
richesse_df = module_region.compute_richesse(df1_fixed, df2_fixed)
richesse_df


,region_id,region,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,11,ÎLE-DE-FRANCE,3.608041,3.780555,3.696251,3.765839,4.035769,3.890366,3.693009,3.994618,4.06881,4.064486
1,24,CENTRE-VAL DE LOIRE,1.475181,1.459593,1.381489,1.334053,1.438393,1.41008,1.3249,1.420395,1.429131,1.433571
2,27,BOURGOGNE-FRANCHE-COMTÉ,1.445978,1.469261,1.401884,1.370449,1.482636,1.459818,1.361757,1.481436,1.478916,1.535093
3,28,NORMANDIE,1.394068,1.408043,1.338704,1.321845,1.41013,1.385802,1.317013,1.411464,1.387228,1.429794
4,32,HAUTS DE FRANCE,1.329440,1.368905,1.292007,1.283720,1.407114,1.359003,1.292271,1.391317,1.379732,1.402894
5,44,GRAND EST,1.592189,1.616233,1.540258,1.535817,1.633375,1.59884,1.517944,1.618347,1.620915,1.668831
6,52,PAYS DE LA LOIRE,1.333234,1.363174,1.295101,1.288082,1.406317,1.398685,1.323323,1.438462,1.430036,1.419568
7,53,BRETAGNE,1.370342,1.403875,1.317954,1.306276,1.433502,1.410507,1.337357,1.468594,1.44057,1.473327
8,75,NOUVELLE AQUITAINE,1.440469,1.469202,1.369579,1.359309,1.471731,1.432206,1.3724,1.495559,1.478492,1.469908
9,76,OCCITANIE,1.444609,1.495388,1.382525,1.383604,1.485129,1.446965,1.366958,1.490175,1.431404,1.421652
